
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>LSTM</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Long Short Time Memory LSTM
1. w 1991 Sepp Hochreiter obronił pracę dyplomową w Monachium w której 
  * przedstawił szczegółową analizę uczenia sieci rekurencyjnych
  * odkrył zjawisko zanikajacego i eksplodujacego gradientu
2. w 1997 zaproponował, wraz z Jurgenem Schmidhuberem, model LSTM 
  * Neural Computation:9(8):1735-1780 
  * wcześniej w 1995 w _technical document_ w Monachium
3. model LSTM stał się początkiem dla wielu innych modeli
  * rozwinięcia LSTM, np. GRU (dodatkowe bramki, inny przepływ, etc.)
  * sieci warstwowe typu Highway
  * sieci ResNet

## LSTM (za Hochreiterem)
1. jako podstawowy problem Hochreiter zauważył ___zanikający sygnał błędu___
  * błąd zanika i sieć nie uczy się niczego w sensownym czasie
  * dla sigmoidalnych funkcji aktywacji wagi musiałyby być większe od $\approx4$ by sygnał miał wystarczającą wartość
  * inicjalizacje wag do wyższych wartości nie pomagają, bo odpowiednia pochodna maleje jeszcze szybciej
  * BPTT jest bardzo czuły na ostatnie zmiany/sygnały
  * Hochreiter zauważył, że konieczne jest zapewnienienie stałego przepływu sygnału błędu
    * wniosek: funkcja aktywacji __musi być liniowa__
2. komórka LSTM

<img src="../nn_figures/LSTM-Hochreiter.pdf" width="80%"> [Neural Computation]

 1. wejście --> 
 2. zgniatanie wejścia --> 
 3. bramkowanie wejścia sygnałem bramki --> 
 4. zapamiętywanie i zapominanie --> 
 5. zgniatanie wyjścia --> 
 6. bramkowanie wyjścia

<img src="../nn_figures/gers_lstm.png" width="80%"> [Hochreiter]

  * dodaje (multiplikatywną) __bramkę wejściową__ $in$, która ma zabezpieczyć zawartość pamięci od wpływu _nieistotnych_ wejść
  * analogicznie __bramkę wyjściową__ $out$ mającą zabezpieczyć inne komórki przed wpływem (aktualnych) nieistotnych informacji w komórce
  * __czemu bramki ?__
    * bramka wejsciowa $in$ kontroluje przepływ sygnału błędu by zabezpieczyć przed konfliktami wag
      * czasem komórka _powinna_ uzyć wejścia z innej komórki
      * czasem nie
      * bramka wejsciowa kontroluje to
    * podobnie bramka $out$ kontroluje wagi wyjściowe
    * bramki wejściowa/wyjściowa muszą się __nauczyć__, które sygnały wyłapać/zablokować

### LSTM przykład (za Hochreiterem)
<img src="../nn_figures/LSTM-Hochreiter-flow.pdf" width="80%"> [Neural Computation]


## LSTM krok po kroku (za Christopher Olah)
### bramka wejściowa

<img src="../nn_figures/LSTM3-focus-f.png" width="80%"> [Colah]
  * sigmoidalna bramka wejściowa decyduje __które__ informacje będą aktualizowane
    * wartość poprzedniej pamieci jest __wymnażana__ przez $f_t$
  * $f_t$ jest bramką _decydującą_ które informacje wejściowe są istotne i należy je zapamiętać
    * pojawia się cudzysłów otwierający, trzeba zapamiętać ten stan
      

### propozycja i aktualizacja stanu
<img src="../nn_figures/LSTM3-focus-i.png" width="80%"> [Colah]
  * komórka decyduje które wartości z wejścia należy _dodać_ do tej aktualizowanej
    * wartości $i_t*\widetilde{C}_t$ są dodawane do poprzednio wyczyszczonej i odpowiednio przeskalowane
    * to decyduje, których informacji z wejścia (i stanu pamięci) należy teraz użyć
  * te wartości połączone decydują o wyjściu komórki $C_t$
  * tutaj porzucane są informacje zapominane
    * pojawia się cudzysłów zamykający, więc zapominamy informację o tym stanie


<img src="../nn_figures/LSTM3-focus-C.png" width="80%"> [Colah]

### wyjście

<img src="../nn_figures/LSTM3-focus-o.png" width="80%"> [Colah]
  * po pierwsze bramka __wyjściowa__ (sigmoidalna) decydująca, które elementy stanu należy przekazać na wyjście $h_t$
  * stan komórki jest reskalowany do $(-1,+1)$ przez $\tanh$
  * przeskalowany stan komórki jest filtrowany przez bramkę wyjściową
  
## warianty
#### bramki wykorzystują wgląd (tzw. peephole) w stan komórki

<img src="../nn_figures/LSTM3-var-peepholes.png" width="80%"> [Colah]
  * wykorzystanie aktualnego stanu na każdym kroku
    * kontekstowa praca poszczególnych bramek
  * nie zawsze wykorzystywane przez wszystkie bramki

#### połączenie bramek zapominającej i wejściowej

<img src="../nn_figures/LSTM3-var-tied.png" width="80%"> [Colah]
  * zapomina jedynie te składniki, w które zostanie wstawiona nowa informacja

#### Gated Recurrent Unit (Cho, 2014)

<img src="../nn_figures/LSTM3-var-GRU.png" width="80%"> [Colah]
  * spore uproszczenie, a przez to staje się popularna
  * połączenie bramek zapiminającej i wejściowej
  * łączy stan komórki $C_t$ wraz ze stanem ukrytym komórki $h_t$

## Jakie parametry?
1. uczenie sieci LSTM nie jest trudne
  * jest bardzo wiele wariantów
  * architektura nie jest oczywista i ma wiele składników możliwe, że na wiarę
  * które są najlepsze i jak je uczyć?
2. Rafał Jozefowicz, Zaremba i Sutskever wykonali duży przegląd możliwych architektur wyciągając szereg wniosków
  * istnieje wiele architektur podobnych do GRU lepszych na wielu zadaniach
  * LSTM: jest generalnie najlepszy jeśli użyty był Dropout
  * LSTM: dodawanie jedynki do bramki zapominającej zwykle minimalizuje różnicę do najlepszych architektur
    * typowa inicjalizacja ustawia wszystkie wagi na małe wartości
    * inicjalizacja biasu bramki zapominajacej na małą wartość sprzyja pojawieniu się zanikającego gradientu
    * rozwiązaniem może być inicjalizacja go na wyższe wartości rzędu $1 - 2$
  * istotność bramek
    * bramka wejściowa __jest__ istotna
    * bramka wyjściowa __nie jest__ istotna
    * bramka zapominająca __jest bardzo__ istotna dla wszystkich problemów __poza__ modelowaniem języka
3. autorzy wykorzystali prostą procedurę przeszukiwania
  * utrzymywania listy 100 znalezionych najlepszych architektur poszukując dla nich najlepszych hiperparametrów
  * w każdym etapie wykonuje jeden z kroków
    * losuje jedną ze 100 architektur
      * ewaluuje 20 losowych ustawień hiperparametrów dla każdego z 3 zadań
      * ocenia przez
      $$\min\frac{\text{najlepsza dokładność dla architektury dla zadania}}{\text{najlepszy wynik GRU dla zadania}}$$
      przy czym GRU były wyliczone dla wszystkich dozwolonych architektur
    * wybiera jedną ze 100 architektur
      * mutuje parametry
  * autorzy
    * ewaluowali 10 tysięcy różnych architektur
    * 1000 z nich przeszło początkowy test zapamiętywania
      * 5 znaków w sekwencji, dla wszystkich 26 możliwości
      * jest czytanych w sekwencji
      * i ma być odtworzone w tej samej sekwencji
    * te 1000 architektur było sprawdzone dla średnio 2200 konfiguracji
    * razem wykonali testy dla ok. 230 tysiecy konfiguracji
    * zadania
      * obliczenie sumy (w postaci znaków) dla sekwencji składajacej się z dwóch sekwencji
      * predykcja następnego znaku w kodzie XML
      * modelowanie języka Penn Tree-Bank
      * (dodatkowo) modelowanie muzyki polifonicznej
      
  * rezultaty
    * GRU było lepsze od LSTM na wszystkich zadaniach poza modelowaniem języka
    * jeśli wykorzystano dropout, to LSTM było zdecydowanie najlepsze dla problemów modelowania jezyka
    * LSTM z wysokim biasem bramki zapominajęcej było lepsze od innych LSTM i GRU na prawie wszystkich zadaniach
    * trzy mutacje znalezione okazały się konkurencyjne do innych modeli

## Wielkość ukrytej warstwy
1. $x_t$ i $h_t$ są wektorami
  * jeśli rozpoznajemy znaki, to $h_t$ będzie wektorem prawdopodobieństw znaków
  * w przypadku słów, trzeba wykorzystać jakiś word-embedding, by zredukować wymiarowość
    * word-embedding pozwoli reprezentować słowa jako wektory w $\mathbb{R}^K$
    * word-embedding może być uczony __w trakcie__ uczenia modelu
2. modele LSTM zawierają pojęcie __hidden layer__
  * warstwa ukryta jest zbiorem neuronów w bramkach zapominajacych, wejsciowych, wyjsciowych, etc.
  * ich wielkość to właśnie warstwa ukryta
  * większa warstwa ukryta zapewnia modelowi większą pojemność
3. innym typowym parametrem jest żądanie by model zwracał całe sekwencje
  * w Keras to parametr `return_sequences`
 
  <img src="../nn_figures/Keras-LSTM-return-sequences.png" width="80%"> [Chollet]
  * to pozwala budować sieci __wielopoziomowe__
  <img src="../nn_figures/LSTM-two-layer.png" width="80%"> [Colah]




# Sieci Highway
1. Dla wielu problemów __głębsze__ sieci dają __lepsze__ wyniki
  * wciąż problemem jest __zanikający gradient__
  * potrzebne nowe modele sieci radzące sobie z tym
  * rozwiązaniem może być wykorzystanie __bramek zapominających__ wziętych z LSTM
    * początkowe rozwiązania Srivastavy były znacznie bardziej oparte na LSTM
    * końcowe jest dużym uproszczeniem
2. różnymi podejściami są
  * lepsze algorytmy uczące, inicjalizacje, lepiej dopasowane funkcje aktywacji
  * połączenia skrótowe
    * zawsze obecne w badaniach połączenie między warstwami
      * przykładem sieć dla XOR
    * połączenie __do__ warstw wyjściowych dodające szum
    * dodatkowe funkcje kosztu w trakcie uczenie (patrz Inception)
2. aktywacja nowej warstwy
  * jest częściowo kopią aktywacji poprzedniej (a więc wejścia)
  * częściowo nowo wyliczoną wartością
  * połączenie jako kombinacja liniowa
  * parametry kombinacji są __uczone__
  $$\begin{align}
  y_i &= T(x, w_T)\,\,\sigma\left(\sum_{j}w_{ij}x_j+b\right)+(1-T(x; w_T))x\\
  T&=\sigma\left(\sum_{j}w_{T_{ij}}x_j+b_T\right)
  \end{align}$$
  gdzie $\sigma()$ jest funkcją logistyczną
  * $T$ to bramka __transfomacji__
  * $1-T$ to bramka __przeniesienia__ (ang. carry)
3. można tak ustawić $b'$, że $T$ będzie zmierzało do zera
  * wtedy aktywacja następnej warstwy jest bliska poprzedniej
  $$y = 
  \begin{cases} x &\mbox{if } T() = 0 \\ 
  \sigma\left(\sum_{j}w_{ij}x_j+b\right) & \mbox{if } T() = 1 
  \end{cases}
  $$
  * informacja może przechodzić przez wiele warstw __bez jej _rozcieńczenia___
  * Srivastava i Schmidhuber nazwali takie ścieżki __information highways__
4. definicja wymaga by warstwy miały __takie same wymiary__!
  * jeśli zmiana wymiaru jest konieczna, wtedy można
    * zamienić wejście $x$ na __rozszerzone__ przez odpowiednie
      * dopełnienie zerami
      * odpowiednie wycięcie fragmentu
    * dodać __zwykłą__ warstwę i później kontynuować warstwy typu highway
  * działa podobnie dla sieci konwolucyjnych
  * super! ale nic za darmo

## uczenie sieci Highway

<img src="../nn_figures/highway-layers.pdf" width="90%"> [Srivastava]
1. aktualizacja wag wykorzystując SGD z momentum
2. inicjalizacja
  * inicjalizacja $w_T, b_T$ __niezależna__ od inicjalizacji $w,b$
  * $b_T$ inicjalizowane na nieduże wartosci ujemne rzędu $-1 \ldots -10$
    * powoduje na początku uczenia zachowanie w kierunku czynnika przeniesienia (carry)
    * z początku większość informacji jest kopiowana
    * analogiczne do sugestii inicjalizacji bramki zapominania w sieciach LSTM
<img src="../nn_figures/highway-weights.pdf" width="90%"> [Srivastava]
2. najlepsze 50 sieci dla MNIST i CIFAR-100
  * zerowa warstwa jest zwykła i zmniejsza wymiar do stałego wymiaru 50
    * kolejno: bias bramki transformacji, średnie wyjście z bramki transformacji (dla 10 tysiecy przykładów), 
    * i dalej: przykładowe wyjscie dla bramki transformacji i całkowite wyjście dla losowego przykładu
  * biasy bramki transformacji __rosną__ wraz z głębokością
* średnie wyjścia bramek transformacji __zanikają__ wraz z głębokością
  * czynnik przeniesienia staje się istotniejszy wraz z odległoscią od wejścia
  
3. istotność warstw w modelu

<img src="../nn_figures/highway-warstwy.pdf" width="80%"> [Srivastava]
  * jaka jest istotność warstw w modelu?
  * uszkodzenie pojeynczej warstwy przez ustawienie bramki transformacji pojedynczej warstwy na 0
    * wtedy sieć kopiuje wejście
    * dla problemu MNIST wyraźnie widać, że od pewnego momentu problem jest już praktycznie rozwiązany
      * pod koniec jest pewna strata, wynikająca zapewne z konieczności zsumowania wyników (ostatnia wyjściowa warstwa to softmax)
    * CIFAR-100 jest znacznie trudniejszy, co wyraźnie widać
  * to pokazuje, że nadmiar warstw __nie jest__ problemem


## Mask RCNN
<img src="../nn_figures/mask-rcnn.pdf" width="70%" align="center"> [He et al.]

1. Region Proposal Network RPN wyszukuje obszary potencjalnie zawierające obiekty
2. w drugim etapie MASK-RCNN
  * przeewiduje klasę
  * bounding box
  * maskę
<img src="../nn_figures/mask-rcnn-result.pdf" width="90%"> [He et al.]



# Sieci rezydualne z losowo usuwanymi warstwami (Huang et al.)

<img src="../nn_figures/highway-warstwy.pdf" width="80%"> [Srivastava]

0. Uczenie bardzo głębokich sieci napotyka na wiele problemów
  * zanikające gradienty
    * wielokrotne wymnażanie (lub konwolucje) przy małych wagach powoduje brak efektywnosci uczenia wag bliskich wejscia
    * potrzebne dobra inicjalizacja, nadzorowane uczenie warstw pośrednich (patrz Inception), Batch Normalization
  * zanikające uzycie cech
    * cechy wejściowe lub wcześnie wyliczone zanikają wskutek wymnażania wag
    * są trudne do uzycia później
    * konieczne połączenia skrótowe między wielu warstwami (patrz DenseNet)
  *  uczenie może być bardzo czasochłonne ze względu na bardzo dużą liczbę parametrów
  * dla wielu problemów wystarczająca jest mniejsza liczba warstw
2. A może by część warstw usunąć?
  * określić szansę __istotności__ warstwy w trakcie uczenia
  * losowo warstwy redukować do identyczności
  * przy inferencji uzywać wszystkich
3. niech $r\sim Bernoulli$
  $$H_k(x_{k-1}) = ReLU(r_k F_k(x_{k-1}) + x_{k-1})$$
  * dla $r_k=1$ mamy zwykły blok rezydualny
  $$H_k(x_{k-1}) = ReLU(F_k(x_{k-1}) + x_{k-1})$$
  * dla $r_k=0$ tylko przeniesienie
  $$H_k(x_{k-1}) = ReLU(x_{k-1})$$
    $$x_{k-1}=H_{k-1}(x_{k-2})=ReLU(r_{k-1} F_k(x_{k-2}) + x_{k-2})$$
    stąd
    $$\begin{align}
    H_k(x_{k-1})=ReLU(x_{k-1})&=ReLU(ReLU(r_{k-1} F_k(x_{k-2}) + x_{k-2}))\\
    &=ReLU(r_{k-1} F_k(x_{k-2}) + x_{k-2})\\
    &=x_{k-1}\\
    &=H_{k-1}(x_{k-1})
    \end{align}$$
4. Autorzy proponują używać __prawdopodobieństwa przeżycia__ $p_k$ dla rozkładu Bernoulliego zmniejszanego liniowo wraz z numerem warstwy
$$p_k=1-\frac{k}{K}(1-p_K)$$

  <img src="../nn_figures/resnet-stochastic.pdf" width="80%"> [Huang et al.]

  * im dalej od wejscia, tym większa szansa na opuszczenie
    * zgodne z obserwacjami w sieciach Highway
  * warstwy opuszczane dla każdego mini-batcha
  * autorzy używają $p_K=0.5$
    * 
    * uczenie okazuje się stabilne
  * możliwe jest użycie stałego $p_k=p_K$
    * zwykle większe przyspieszenie (czemu???)
    * jednak zdecydowanie słabsze wyniki
      * dopiero wysoka wartość $p_K$ daje znaczący spadek błędu testowania, jednak porównywalny dla tego z liniowym wzrostem $p_k$ dla znacznie mnieszego $p_K$
    * $p_K=0.5$ daje najlepsze wyniki dla reguły liniowej
    * a czy możliwe jest __adaptacyjne__ modyfikowanie $p_k$???
      * warstwy blisko końcowej mają większe znaczenie
      * rozpocząć do reguły liniowej
      * co ustaloną liczbę kroków wykonywać test amputacji warstw
      * odpowiednio zwiększać/zmniejszać $p_k$
5. Inferencja
  * w trakcie poszczególne $F_k()$ są wykorzystywane __losowo__ proporcjonalnie do $p_k$
  * inferencja wymaga przewtorzenia 
  $$\widehat{H}_k(\widehat{x}_{k-1}) = ReLU(p_k F_k(\widehat{x}_{k-1}) + \widehat{x}_{k-1})$$
5. efekty?
  * lepsze wyniki 
    * błąd na zbiorze trenującym jest __wyższy__ niż ten dla pełnej sieci
    * bład na zbiorze testujacym jest __niższy__ dając jedne z [najlepszych znanych wyników](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)
      * dla CIFAR-10 $5.25\%$ o ponad $1\%$ (ResNet-110)
      * dla CIFAR-100 $24.98\%$ o prawie $3\%$ (ResNet-110)
  * efekty dla ekstremalnie głębokich sieci
    * ResNet o głębokości $1202$ warstw dawał wyniki __słabsze__ niż ResNet-110 dla CIFAR-10
      * overfitting
    * stochastyczne usuwanie warstw na takiej sieci dało __spadek__ błędu ($4.91\%$, spadek o ok. $0.3\%$)
  * znaczne przyspieszenie uczenia
    * czas uczenia jest proporcjonalny do __oczekiwanej__ głębokości sieci, tzn. liczby pozostawionych warstw
      $$\mathbb{E}(\overline{K})=\sum_{k=1}^Kp_k$$
    * dla $p_K=0.5$ mamy $$\mathbb{E}(\overline{K})\simeq3/4K$$
  * dobry wpływ na wielkość gradientu
    * norma gradientu sieci stochastycznej była stale większa od normy dla sieci o stałej długości
  * efekty przypominając Dropout
    * jednak usuwane są __warstwy__, nie pojedyncze neurony
    * generowanych jest losowych $2^K$ sieci